In [6]:
######################################################################################################################################################
## CLIENT PYTHON CODE BELOW ALLOWS YOU TO VISUALIZE HOW WILL JSON RESPONSE RETURNED BY THE PYTHON SERVER WILL LOOK LIKE IF PLOTTED ON A PYTHON MAP. ##
## FTSP TASK: IMPLEMENT A CLIENT MOBILE APP PAGE THAT MAKE A REQUEST TO THE PYTHON SERVER AND DISPLAY THE RESULTS ON FLUTTER MAP.                   ##
## THE RESULTS ON FLUTTER MAP SHOULD LOOK SIMILAR TO THOSE OF A PYTHON MAP                                                                          ##
## IT SHOULD HAVE START POINT, END POINT AND THE POIS IN BETWEEN THE START POINT AND END POINT                                                      ##
## YOU MAY MAKE A REQUEST TO THE SERVER USING DIFFERENT COMBINATION OF topn(1,2) + vehiclemode(1) + mode(1,2,3) TO SEE HOW THE MAP CHANGES          ##
######################################################################################################################################################
# !pip install folium.plugins
# !pip install pyarrow

In [7]:
import datetime
import base64
import requests
import uuid
import time
import glob, random
import json
import pandas as pd
import webbrowser

from IPython.core.display import display, HTML
display(HTML("<style>.output_scroll { height: unset !important; }</style>"))

def convert_to_minutes(time_str):
    hours, minutes = time_str.split('h ')
    hours = int(hours)
    minutes = int(minutes[:-3])
    total_minutes = hours * 60 + minutes
    return total_minutes
def convert_to_time(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    time_str = f"{hours}h {minutes}min"
    return time_str
def convert_hex_to_decimal(string):
    hex_values = string.split(':')
    hex_value = hex_values[1]
    if hex_value.startswith('0x'):
        decimal_value = int(hex_value[2:], 16)
        return decimal_value
    return None
def convert_to_seconds(hours, minutes):
    total_seconds = (hours * 3600) + (minutes * 60)
    return total_seconds   

def convert_user_avail_time(user_avail_time):
    # Convert seconds to hours and minutes
    hours = user_avail_time // 3600
    minutes = (user_avail_time % 3600) // 60
    # Return the user available time in hours and minutes
    return f"{hours}h {minutes}min"
def transportation_mode(number):
    if number == 1:
        return "Walk"
    elif number == 2:
        return "Drive"
    elif number == 3:
        return "Cycle"
    elif number == 4:
        return "Public Transport"
    else:
        return "invalid number"
def recommendation_mode(number):
    if number == 1:
        return "No return to start point"
    elif number == 2:
        return "Return to start point" #Pad start point to end of itinerary
    elif number == 3:
        return "End point specified by user" #Pad user specified point to end of itinerary
    else:
        return "invalid number" 

def process_POI(string_list):
    output = []
    for item in string_list:
        output.append(item)
    output1 = str(output)
    output1 = output1.replace("[]","None")
    return output1

def AddAnimation(d_route, color, map_sg, tooltip):
    import folium
    import folium.plugins as plugins
    ant_path = folium.plugins.AntPath(
        locations=d_route,
        color=color,
        weight=20,
        delay=500,  # Delay between ant movement steps in milliseconds
        dash_array=[10, 20],  # Dash array for the ant's trail
        pulse_color=color,
        tooltip=tooltip
    )
    ant_path.add_to(map_sg)    

def drawMapModes1234Capable(df, index, lat, long):
    import folium
    import polyline
    import random

    map_sg = folium.Map(location=[lat, long], zoom_start=15)

    row = df.iloc[index]
    routes, itineraries, categories, StartTimePOIs, ReachTimePOIs, LeaveTimePOIs = row['Routes'], row['Itinerary'], row['POICategories'], row['StartTimePOI'], row['ReachTimePOI'], row['LeaveTimePOI']
    color_mapping = {}
    if len(routes) > 1 and 'route_summary' in routes[1] and 'start_point' in routes[1]['route_summary']:
        sp = routes[1]['route_summary']['start_point']
        sp_name = "(START POINT) " + "<br>" + str(itineraries[0]) + "<br>" + "Category/SubCategory/SubSubCategory: "+ str(process_POI(categories[0])) + "<br>" + "StartTime: " + str(StartTimePOIs[0])
        ep_name = "(POI 1) " + "<br>" + str(itineraries[1]) + "<br>" + "Category/SubCategory/SubSubCategory: "+ str(process_POI(categories[1])) + "<br>" + "ArrivalTime: " + str(ReachTimePOIs[0]) + "<br>" + "LeaveTime: " + str(LeaveTimePOIs[0])
        d_route = polyline.decode(routes[0]['route_geometry'])
        color = '#{:06x}'.format(random.randint(0, 0x080808))

        folium.PolyLine(locations=d_route, color=color, weight=5, opacity=0.7).add_to(map_sg)
        folium.RegularPolygonMarker(location=d_route[0], number_of_sides=4, radius=22, rotation=45, color=color, fill_color=color, fill_opacity=1, tooltip=sp_name).add_to(map_sg)
        folium.RegularPolygonMarker(location=d_route[-1], number_of_sides=4, radius=22, rotation=45, color=color, fill_color=color, fill_opacity=1, tooltip=ep_name).add_to(map_sg)

        AddAnimation(d_route, color, map_sg, "")

        for n in routes[0]['route_instructions']:
            n_text, n_location = n[9], n[3].split(',')
            folium.Marker(location=n_location, icon=folium.Icon(color='gray'), tooltip=n_text).add_to(map_sg)

        for index, route in enumerate(routes[1:], start=1):
            route_geometry, route_instructions, route_name, sp, ep = route['route_geometry'], route['route_instructions'], route['route_name'], route['route_summary']['start_point'], route['route_summary']['end_point']
            sp_name = itineraries[index] + "<br>" + str(process_POI(categories[index])) + "<br>" + "ArrivalTime: " + str(ReachTimePOIs[index]) + "<br>" + "LeaveTime: " + str(LeaveTimePOIs[index])
            if int(index) == int(len(itineraries)-9%7):
                ep_name = "(END POINT) " + "<br>" + str(itineraries[index + 1]) + "<br>" + "Category/SubCategory/SubSubCategory: " +  str(process_POI(categories[index + 1])) + "<br>" + "ArrivalTime: " + str(ReachTimePOIs[index]) + "<br>" + "LeaveTime: " + str(LeaveTimePOIs[index])
            else:
                ep_name = "(POI " + str(index+1) + ") " + str(itineraries[index + 1]) + "<br>" + "Category/SubCategory/SubSubCategory: "+ str(process_POI(categories[index + 1])) + "<br>" + "ArrivalTime: " + str(ReachTimePOIs[index]) + "<br>" + "LeaveTime: " + str(LeaveTimePOIs[index])

            if (sp, ep) in color_mapping:
                color = color_mapping[(sp, ep)]
            else:
                color = '#{:06x}'.format(random.randint(0, 0x080808))
                color_mapping[(sp, ep)] = color

            d_route = polyline.decode(route_geometry)
            folium.PolyLine(locations=d_route, color=color, weight=5, opacity=1).add_to(map_sg)
            folium.Marker(location=d_route[0], icon=folium.Icon(color=color), tooltip=sp_name).add_to(map_sg)
            folium.RegularPolygonMarker(location=d_route[-1], number_of_sides=4, radius=22, rotation=45, color=color, fill_color=color, fill_opacity=1, tooltip=ep_name).add_to(map_sg)

            AddAnimation(d_route, color, map_sg, "")  

            for n in route_instructions:
                n_text, n_location = n[9], n[3].split(',')
                folium.Marker(location=n_location, icon=folium.Icon(color='gray'), tooltip=n_text).add_to(map_sg)
            
        return map_sg
    else:
        # Extracting information from the dataframe
        routes_str = df.loc[index, 'Routes']
        routes_dict = routes_str

        # Creating a map centered at the origin
        m = folium.Map(location=[lat, long], zoom_start=15)
        # m = folium.Map(location=[routes_dict[0]['plan']['from']['lat'], routes_dict[0]['plan']['from']['lon']], zoom_start=15)

        # Iterate over each route and draw the polyline
        for i, route in enumerate(routes_dict):
            origin = route['plan']['from']
            destination = route['plan']['to']
            waypoints = route['plan']['itineraries'][0]['legs']

            # Adding markers for the origin and destination with tooltips
            # folium.Marker([origin['lat'], origin['lon']], tooltip=origin['name']).add_to(m)
            # folium.Marker([destination['lat'], destination['lon']], tooltip=destination['name']).add_to(m)
            if i == 0:
                folium.Marker([origin['lat'], origin['lon']], tooltip="(Start Point) " + route['plan']['itineraries'][0]['legs'][0]['to']['name']).add_to(m)
            else:
                folium.Marker([origin['lat'], origin['lon']], tooltip=route['plan']['itineraries'][0]['legs'][0]['to']['name']).add_to(m)

            if i == len(routes_dict) - 1:
                folium.Marker([destination['lat'], destination['lon']], tooltip="(End Point)").add_to(m)

            # Decode and add polylines
            for waypoint in waypoints:

                encoded_polyline = waypoint['legGeometry']['points']
                decoded_polyline = polyline.decode(encoded_polyline)

                # Generate a random color
                color = "#{:06x}".format(random.randint(0, 0x111111))

                # Get the mode of transportation for the tooltip
                mode = waypoint['mode']

                # Add polyline to represent the route segment with random color
                folium.PolyLine(decoded_polyline, color=color, weight=2.5, opacity=1, tooltip=mode).add_to(m)

                # Add Animation
                AddAnimation(decoded_polyline, color, m, mode)

        # Display the map
        return m        

# def replace_backslashes(filename):
#     with open(filename, 'r') as file:
#         content = file.read()
    
#     updated_content = content.replace('\\\\', '\\')

#     with open(filename, 'w') as file:
#         file.write(updated_content)    

In [8]:
##########
## Demo ##
##########
# User 108938050734323740126 -> Mostly vegetarian diet
from IPython.display import display, HTML, Javascript
import csv
UserAvailHour = 8                      # hours available for user
UserAvailMin = 0                       # minutes available for user
json_data_to_post = {
#'uid': "108938050734323740126",        # uid (21 character string)
'uid': "108938050734323740126",        # uid (21 character string)
'topn': "3",                           # itineraries to generate
# 'topk': "0",                         # pois per itinerary
'useravailtime': convert_to_seconds(UserAvailHour, UserAvailMin),# user available time in seconds
    
'vehiclemode': "1",                    # 1 = walk, 2 = drive, 3 = cycle, 4 = public transport
'mode': "1",                           # 1 = no return to start point, #2 = return to start point, #3 user specified end point
   
'userendpoi': "Clementi Mall",                   # get placeid given name of place
'latuser': "1.2931",                   # user current latitude (CITY HALL)
'longuser': "103.8520",                # user current longitude (CITY HALL)
'allow_consecutive_foodpoi_in_itinerary': True,
'StartDateTime': datetime.datetime.now().strftime("2023-09-22 10:00:00"),
'request_model_to_predict_ranked_pois_given_uid': "300"          # For debugging purposes
}

# 1.2931, 103.8520
# 1.354410, 103.769156
# 1.332100, 103.954550

try:
  api_url = "http://127.0.0.1:7687/getrecommendation"
  # api_url = "https://poirecommender1.azurewebsites.net/getrecommendation"
  response = requests.post(api_url, json=json_data_to_post)
  response_json_str = json.dumps(response.json(), ensure_ascii=False, indent=4)
  response_data = json.loads(response_json_str)
  
  print("####################")
  print("## Recommendation ##")
  print("####################")    
  print("User: " + json_data_to_post["uid"])    
  # print("POIs per itinerary: " + json_data_to_post["topk"])       
  print("Itineraries: " + json_data_to_post["topn"])       
  print("User Available Time: " + convert_user_avail_time(int(json_data_to_post["useravailtime"])))
  print("Transportation Mode: " + transportation_mode(int(json_data_to_post["vehiclemode"])))           
  print("Recommendation Mode: " + recommendation_mode(int(json_data_to_post["mode"])))           
  print("User Startpoint (Current GPS Location): " + json_data_to_post["latuser"] + "," + json_data_to_post["longuser"])    
  if int(json_data_to_post["mode"])==3:
      print("User Endpoint: " + json_data_to_post["userendpoi"])
  print("allow_consecutive_foodpoi_in_itinerary: " + str(json_data_to_post["allow_consecutive_foodpoi_in_itinerary"]))
  print("StartDateTime: " + json_data_to_post["StartDateTime"])   

  df = pd.DataFrame(response_data)
  desired_order = ['index', 'Itinerary', 'POICategories', 'VehicleMode', 'Coords', 'StartTimePOI', 'ReachTimePOI', 'LeaveTimePOI', 'TotalTime', 'Total distance(km)', 'Routes', 'user_exist_in_db']
  df = df[desired_order]

  df['TotalTime'] = df['TotalTime'].apply(convert_to_minutes)
  df = df.sort_values('TotalTime')
  df['TotalTime'] = df['TotalTime'].apply(convert_to_time)
  df = df.reset_index(drop=True)
  df['index'] = range(len(df))
  
   
#  ##########################
#  ## Write Output to JSON ##
#  ##########################
#  if json_data_to_post["vehiclemode"] == "1":
#      df.to_json('formatforwalkdrivecycle.json', orient='records')
#  elif json_data_to_post["vehiclemode"] == "2":
#      df.to_json('formatforwalkdrivecycle.json', orient='records')
#  elif json_data_to_post["vehiclemode"] == "3":
#      df.to_json('formatforwalkdrivecycle.json', orient='records')
#  elif json_data_to_post["vehiclemode"] == "4":
#      df.to_json('formatforpublictransport.json', orient='records')
       
  display(HTML(df.to_html(index=False)))
          
except Exception as e:
  print("ERROR")
  print(e)

####################
## Recommendation ##
####################
User: 108938050734323740126
Itineraries: 3
User Available Time: 8h 0min
Transportation Mode: Walk
Recommendation Mode: No return to start point
User Startpoint (Current GPS Location): 1.2931,103.8520
allow_consecutive_foodpoi_in_itinerary: True
StartDateTime: 2023-09-22 10:00:00


In [9]:
########################
## Plot On Python Map ##
########################
drawWhichItinerary = 0 #0 means draw first itinerary. 1 means draw second itinerary.
map_sg = drawMapModes1234Capable(df, drawWhichItinerary, json_data_to_post['latuser'],json_data_to_post['longuser'])
map_sg

<ipython-input-7-24993f6d41c3>:128: UserWarning: color argument of Icon should be one of: {'green', 'beige', 'darkgreen', 'cadetblue', 'purple', 'lightgreen', 'white', 'lightblue', 'lightred', 'blue', 'red', 'darkblue', 'black', 'darkred', 'orange', 'gray', 'pink', 'darkpurple', 'lightgray'}.
  folium.Marker(location=d_route[0], icon=folium.Icon(color=color), tooltip=sp_name).add_to(map_sg)
